In [3]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data_summary as rd
# import main
import config
import queries
import uuid
from pandasql import sqldf
from sqlalchemy.dialects.postgresql import insert

files = r"C:\Users\MB27{}5851\Desktop\Temp\rsa_traffic_counts\SMEC RSA Files_GP PRM Sites_Dec21toFeb22\{}337_2{}22{}131.RSA"
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB27{}5851\Desktop\Temp\SYNTELL-test.csv"

In [4]:
GET_HSWIM_HEADER_IDS = """SELECT DISTINCT header_id from trafc.electronic_count_data_type_10 t10 
    where not exists (select header_id from trafc.electronic_count_header_hswim h where h.header_id = t10.header_id)
    and 
    (
        exists (select header_id from trafc.traffic_e_type10_wheel_mass m where m.type10_id = t10.data_id)
        or
        exists (select header_id from trafc.traffic_e_type10_axle_group_mass m where m.type10_id = t10.data_id)
        or
        exists (select header_id from trafc.traffic_e_type10_axle_mass m where m.type10_id = t10.data_id)
    )
    ;"""

HEADER_COLUMNS_QRY = """select * from trafc.electronic_count_header limit 1"""

In [5]:
header_columns = list(pd.read_sql_query(HEADER_COLUMNS_QRY,config.ENGINE).columns)

In [6]:
data = pd.read_sql_query(QRY_TEST, config.ENGINE)

NameError: name 'QRY_TEST' is not defined

In [7]:
def wim_stations_header_insert_qrys(header_id):
    SELECT_TYPE10_QRY = f"""SELECT * FROM trafc.electronic_count_data_type_10 t10
        left join traf_lu.vehicle_classes_scheme_08 c on c.id = t10.vehicle_class_code_primary_scheme
        where t10.header_id = '{header_id}'
        """
    AXLE_SPACING_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        axs.axle_spacing_number,
        axs.axle_spacing_cm
        FROM trafc.electronic_count_data_type_10 t10
        inner join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
        where t10.header_id = '{header_id}'
        """
    WHEEL_MASS_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        wm.wheel_mass_number,
        wm.wheel_mass,
        vm.kg as vehicle_mass_limit_kg,
        sum(wm.wheel_mass*2) over(partition by t10.id) as gross_mass
        FROM trafc.electronic_count_data_type_10 t10
        inner join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
        inner join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
        where t10.header_id = '{header_id}'
        """
    return SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY

In [8]:
def wim_stations_header_insert_dfs(SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY):
    df = pd.read_sql_query(SELECT_TYPE10_QRY,config.ENGINE)
    df = df.fillna(0)
    df2 = pd.read_sql_query(AXLE_SPACING_SELECT_QRY,config.ENGINE)
    df2 = df2.fillna(0)
    df3 = pd.read_sql_query(WHEEL_MASS_SELECT_QRY,config.ENGINE)
    df3 = df3.fillna(0)
    return df, df2, df3

In [9]:
def wim_stations_header_insert(header_id, df, df2, df3):
    try:
        egrl_percent = round((((df.loc[df['edit_code']==2].count()[0])/(df.count()[0]))*100),0) 
    except:
        egrl_percent = 0
    try:
        egrl_percent_positive_direction = round(((df.loc[(df['edit_code']==2)&(df['direction']=='P')].count()[0]/df.loc[df['direction']=='P'].count()[0])*100),0) 
    except:
        egrl_percent_positive_direction = 0
    try:
        egrl_percent_negative_direction = round(((df.loc[(df['edit_code']==2)&(df['direction']=='P')].count()[0]/df.loc[df['direction']=='N'].count()[0])*100),0)  
    except:
        egrl_percent_negative_direction = 0
    try:
        egrw_percent = round((((df2.loc[df2['edit_code']==2].count()[0]+df3.loc[df3['edit_code']==2].count()[0])/df.count()[0])*100),0)   
    except:
        egrw_percent = 0
    try:
        egrw_percent_positive_direction = round((((df2.loc[(df2['edit_code']==2)&(df2['direction']=='P')].count()[0]+df3.loc[(df3['edit_code']==2)&(df3['direction']=='P')].count()[0])/df.loc[df['direction']=='P'].count()[0])*100),0)  
    except:
        egrw_percent_positive_direction = 0
    try:
        egrw_percent_negative_direction = round((((df2.loc[(df2['edit_code']==2)&(df2['direction']=='N')].count()[0]+df3.loc[(df3['edit_code']==2)&(df3['direction']=='N')].count()[0])/df.loc[df['direction']=='N'].count()[0])*100),0)   
    except:
        egrw_percent_negative_direction = 0
    num_weighed = df3.groupby(pd.Grouper(key='id')).count().count()[0] or 0 
    num_weighed_positive_direction = df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id')).count().count()[0] or 0  
    num_weighed_negative_direction = df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id')).count().count()[0] or 0  
    mean_equivalent_axle_mass = round((df3.groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean(),2) or 0
    mean_equivalent_axle_mass_positive_direction = round((df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean(),2) or 0
    mean_equivalent_axle_mass_negative_direction = round((df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean(),2) or 0
    mean_axle_spacing = round((df2.groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'],0) or 0
    mean_axle_spacing_positive_direction = round((df2.loc[df2['direction']=='P'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'],0) or 0
    mean_axle_spacing_negative_direction = round((df2.loc[df2['direction']=='N'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'],0) or 0
    e80_per_axle = round(((df3['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    e80_per_axle_positive_direction = round(((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    e80_per_axle_negative_direction = round(((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    olhv = len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique()) or 0
    olhv_positive_direction = len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique()) or 0
    olhv_negative_direction = len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique()) or 0
    try:
        olhv_percent = round(((len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique())/len(df3['id'].unique()))*100),2)
    except ZeroDivisionError:
        olhv_percent = 0
    try:
        olhv_percent_positive_direction = round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique())/len(df3.loc[df3['direction']=='P']['id'].unique()))*100),2)
    except ZeroDivisionError:
        olhv_percent_positive_direction = 0
    try:
        olhv_percent_negative_direction = round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique())/len(df3.loc[df3['direction']=='N']['id'].unique()))*100),2)
    except ZeroDivisionError:
        olhv_percent_negative_direction = 0
    tonnage_generated = ((df3['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    tonnage_generated_positive_direction = ((df3.loc[df3['direction']=='P']['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    tonnage_generated_negative_direction = ((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    olton = (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum().astype(int) or 0
    olton_positive_direction = (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum().astype(int) or 0
    olton_negative_direction = (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum().astype(int) or 0
    try:
        olton_percent = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum().round(2)/(df3['wheel_mass']*2).sum().round(2))*100,2)
    except ZeroDivisionError:
        olton_percent = 0
    try:
        olton_percent_positive_direction = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum().round(2)/(df3.loc[df3['direction']=='P']['wheel_mass']*2).sum().round(2))*100,2)
    except ZeroDivisionError:
        olton_percent_positive_direction = 0
    try:
        olton_percent_negative_direction = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum().round(2)/(df3.loc[df3['direction']=='N']['wheel_mass']*2).sum().round(2))*100,2)
    except ZeroDivisionError:
        olton_percent_negative_direction = 0
    ole80 = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    ole80_positive_direction = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    ole80_negative_direction = round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum(),0) or 0
    try:
        ole80_percent = round((round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum(),0)/round(round(((df3['wheel_mass']*2/8200)**4.2).sum(),0)))*100,2)
    except ZeroDivisionError:
        ole80_percent = 0
    try:
        ole80_percent_positive_direction = ((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2)
    except ZeroDivisionError:
        ole80_percent_positive_direction = 0
    try:
        ole80_percent_negative_direction = ((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2)
    except ZeroDivisionError:
        ole80_percent_negative_direction = 0
    xe80 = round(((df3['wheel_mass']*2/8200)**4.2).sum()-(((df3['wheel_mass']*2*0.05)/8200)**4.2).sum(), 2) or 0
    xe80_positive_direction = round(((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum()-(((df3.loc[df3['direction']=='P']['wheel_mass']*2*0.05)/8200)**4.2).sum(),2) or 0
    xe80_negative_direction = round(((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum()-(((df3.loc[df3['direction']=='N']['wheel_mass']*2*0.05)/8200)**4.2).sum(),2) or 0
    try:
        xe80_percent = (((((df3['wheel_mass']*2/8200)**4.2).sum()-(((df3['wheel_mass']*2*0.05)/8200)**4.2).sum())/((df3['wheel_mass']*2/8200)**4.2).sum())*100).round()
    except ZeroDivisionError:
        xe80_percent = 0
    try:
        xe80_percent_positive_direction = (((((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum()-(((df3.loc[df3['direction']=='P']['wheel_mass']*2*0.05)/8200)**4.2).sum())/((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum())*100).round()
    except ZeroDivisionError:
        xe80_percent_positive_direction = 0
    try:
        xe80_percent_negative_direction = (((((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum()-(((df3.loc[df3['direction']=='N']['wheel_mass']*2*0.05)/8200)**4.2).sum())/((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum())*100).round()
    except ZeroDivisionError:
        xe80_percent_negative_direction = 0
    try:
        e80_per_day = ((((df3['wheel_mass']*2/8200)**4.2).sum().round()/df3.groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_day = 0
    try:
        e80_per_day_positive_direction = ((((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_day_positive_direction = 0
    try:
        e80_per_day_negative_direction = ((((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_day_negative_direction = 0
    try:
        e80_per_heavy_vehicle = ((((df3.loc[df3['vehicle_class_code_primary_scheme']>3]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['vehicle_class_code_primary_scheme']>3].count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_heavy_vehicle = 0
    try:
        e80_per_heavy_vehicle_positive_direction = ((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')].count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_heavy_vehicle_positive_direction = 0
    try:
        e80_per_heavy_vehicle_negative_direction = ((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')].count()[0])*100).round(2)
    except ZeroDivisionError:
        e80_per_heavy_vehicle_negative_direction = 0
    worst_steering_single_axle_cnt = 0
    worst_steering_single_axle_olhv_perc = 0
    worst_steering_single_axle_tonperhv = 0
    worst_steering_double_axle_cnt = 0
    worst_steering_double_axle_olhv_perc = 0
    worst_steering_double_axle_tonperhv = 0
    worst_non_steering_single_axle_cnt = 0
    worst_non_steering_single_axle_olhv_perc = 0
    worst_non_steering_single_axle_tonperhv = 0
    worst_non_steering_double_axle_cnt = 0
    worst_non_steering_double_axle_olhv_perc = 0
    worst_non_steering_double_axle_tonperhv = 0
    worst_triple_axle_cnt = 0
    worst_triple_axle_olhv_perc = 0
    worst_triple_axle_tonperhv = 0
    bridge_formula_cnt = round((18000 + 2.1 * (df2.loc[df2['axle_spacing_number']>1].groupby('id')['axle_spacing_cm'].sum().mean())),2) or 0
    bridge_formula_olhv_perc = 0
    bridge_formula_tonperhv = 0
    gross_formula_cnt = 0
    gross_formula_olhv_perc = 0
    gross_formula_tonperhv = 0
    total_avg_cnt = df.loc[df['group']=='Heavy'].count()[0]
    total_avg_olhv_perc = 0
    total_avg_tonperhv = round(((df3['wheel_mass']*2).sum()/1000)/df.loc[df['group']=='Heavy'].count()[0],2)
    worst_steering_single_axle_cnt_positive_direciton = 0
    worst_steering_single_axle_olhv_perc_positive_direciton = 0
    worst_steering_single_axle_tonperhv_positive_direciton = 0
    worst_steering_double_axle_cnt_positive_direciton = 0
    worst_steering_double_axle_olhv_perc_positive_direciton = 0
    worst_steering_double_axle_tonperhv_positive_direciton = 0
    worst_non_steering_single_axle_cnt_positive_direciton = 0
    worst_non_steering_single_axle_olhv_perc_positive_direciton = 0
    worst_non_steering_single_axle_tonperhv_positive_direciton = 0
    worst_non_steering_double_axle_cnt_positive_direciton = 0
    worst_non_steering_double_axle_olhv_perc_positive_direciton = 0
    worst_non_steering_double_axle_tonperhv_positive_direciton = 0
    worst_triple_axle_cnt_positive_direciton = 0
    worst_triple_axle_olhv_perc_positive_direciton = 0
    worst_triple_axle_tonperhv_positive_direciton = 0
    bridge_formula_cnt_positive_direciton = round((18000 + 2.1 * (df2.loc[(df2['axle_spacing_number']>1)&(df2['direction']=='P')].groupby('id')['axle_spacing_cm'].sum().mean())),2) or 0
    bridge_formula_olhv_perc_positive_direciton = 0
    bridge_formula_tonperhv_positive_direciton = 0
    gross_formula_cnt_positive_direciton = 0
    gross_formula_olhv_perc_positive_direciton = 0
    gross_formula_tonperhv_positive_direciton = 0
    total_avg_cnt_positive_direciton = df.loc[(df['group']=='Heavy')&(df['direction']=='P')].count()[0]
    total_avg_olhv_perc_positive_direciton = 0
    total_avg_tonperhv_positive_direciton = round(((df3.loc[df3['direction']=='P']['wheel_mass']*2).sum()/1000)/df.loc[df['group']=='Heavy'].count()[0],2)
    worst_steering_single_axle_cnt_negative_direciton = 0
    worst_steering_single_axle_olhv_perc_negative_direciton = 0
    worst_steering_single_axle_tonperhv_negative_direciton = 0
    worst_steering_double_axle_cnt_negative_direciton = 0
    worst_steering_double_axle_olhv_perc_negative_direciton = 0
    worst_steering_double_axle_tonperhv_negative_direciton = 0
    worst_non_steering_single_axle_cnt_negative_direciton = 0
    worst_non_steering_single_axle_olhv_perc_negative_direciton = 0
    worst_non_steering_single_axle_tonperhv_negative_direciton = 0
    worst_non_steering_double_axle_cnt_negative_direciton = 0
    worst_non_steering_double_axle_olhv_perc_negative_direciton = 0
    worst_non_steering_double_axle_tonperhv_negative_direciton = 0
    worst_triple_axle_cnt_negative_direciton = 0
    worst_triple_axle_olhv_perc_negative_direciton = 0
    worst_triple_axle_tonperhv_negative_direciton = 0
    bridge_formula_cnt_negative_direciton = round((18000 + 2.1 * (df2.loc[(df2['axle_spacing_number']>1)&(df2['direction']=='P')].groupby('id')['axle_spacing_cm'].sum().mean())),2) or 0
    bridge_formula_olhv_perc_negative_direciton = 0
    bridge_formula_tonperhv_negative_direciton = 0
    gross_formula_cnt_negative_direciton = 0
    gross_formula_olhv_perc_negative_direciton = 0
    gross_formula_tonperhv_negative_direciton = 0
    total_avg_cnt_negative_direciton = df.loc[(df['group']=='Heavy')&(df['direction']=='N')].count()[0]
    total_avg_olhv_perc_negative_direciton = 0
    total_avg_tonperhv_negative_direciton = round(((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000)/df.loc[df['group']=='Heavy'].count()[0],2)

    UPSERT_STRING = f"""
    insert into trafc.electronic_count_header_hswim (
        header_id,
        egrl_percent,
        egrl_percent_positive_direction,
        egrl_percent_negative_direction,
        egrw_percent ,
        egrw_percent_positive_direction ,
        egrw_percent_negative_direction ,
        num_weighed,
        num_weighed_positive_direction,
        num_weighed_negative_direction,
        mean_equivalent_axle_mass,
        mean_equivalent_axle_mass_positive_direction,
        mean_equivalent_axle_mass_negative_direction,
        mean_axle_spacing,
        mean_axle_spacing_positive_direction,
        mean_axle_spacing_negative_direction,
        e80_per_axle,
        e80_per_axle_positive_direction,
        e80_per_axle_negative_direction,
        olhv,
        olhv_positive_direction,
        olhv_negative_direction,
        olhv_percent,
        olhv_percent_positive_direction,
        olhv_percent_negative_direction,
        tonnage_generated,
        tonnage_generated_positive_direction,
        tonnage_generated_negative_direction,
        olton,
        olton_positive_direction,
        olton_negative_direction,
        olton_percent,
        olton_percent_positive_direction,
        olton_percent_negative_direction,
        ole80,
        ole80_positive_direction,
        ole80_negative_direction,
        ole80_percent,
        ole80_percent_positive_direction,
        ole80_percent_negative_direction,
        xe80,
        xe80_positive_direction,
        xe80_negative_direction,
        xe80_percent,
        xe80_percent_positive_direction,
        xe80_percent_negative_direction,
        e80_per_day,
        e80_per_day_positive_direction,
        e80_per_day_negative_direction,
        e80_per_heavy_vehicle,
        e80_per_heavy_vehicle_positive_direction,
        e80_per_heavy_vehicle_negative_direction,
        worst_steering_single_axle_cnt,
        worst_steering_single_axle_olhv_perc,
        worst_steering_single_axle_tonperhv,
        worst_steering_double_axle_cnt,
        worst_steering_double_axle_olhv_perc,
        worst_steering_double_axle_tonperhv,
        worst_non_steering_single_axle_cnt,
        worst_non_steering_single_axle_olhv_perc,
        worst_non_steering_single_axle_tonperhv,
        worst_non_steering_double_axle_cnt,
        worst_non_steering_double_axle_olhv_perc,
        worst_non_steering_double_axle_tonperhv,
        worst_triple_axle_cnt,
        worst_triple_axle_olhv_perc,
        worst_triple_axle_tonperhv,
        bridge_formula_cnt,
        bridge_formula_olhv_perc,
        bridge_formula_tonperhv,
        gross_formula_cnt,
        gross_formula_olhv_perc,
        gross_formula_tonperhv,
        total_avg_cnt,
        total_avg_olhv_perc,
        total_avg_tonperhv,
        worst_steering_single_axle_cnt_positive_direciton,
        worst_steering_single_axle_olhv_perc_positive_direciton,
        worst_steering_single_axle_tonperhv_positive_direciton,
        worst_steering_double_axle_cnt_positive_direciton,
        worst_steering_double_axle_olhv_perc_positive_direciton,
        worst_steering_double_axle_tonperhv_positive_direciton,
        worst_non_steering_single_axle_cnt_positive_direciton,
        worst_non_steering_single_axle_olhv_perc_positive_direciton,
        worst_non_steering_single_axle_tonperhv_positive_direciton,
        worst_non_steering_double_axle_cnt_positive_direciton,
        worst_non_steering_double_axle_olhv_perc_positive_direciton,
        worst_non_steering_double_axle_tonperhv_positive_direciton,
        worst_triple_axle_cnt_positive_direciton,
        worst_triple_axle_olhv_perc_positive_direciton,
        worst_triple_axle_tonperhv_positive_direciton,
        bridge_formula_cnt_positive_direciton,
        bridge_formula_olhv_perc_positive_direciton,
        bridge_formula_tonperhv_positive_direciton,
        gross_formula_cnt_positive_direciton,
        gross_formula_olhv_perc_positive_direciton,
        gross_formula_tonperhv_positive_direciton,
        total_avg_cnt_positive_direciton,
        total_avg_olhv_perc_positive_direciton,
        total_avg_tonperhv_positive_direciton,
        worst_steering_single_axle_cnt_negative_direciton,
        worst_steering_single_axle_olhv_perc_negative_direciton,
        worst_steering_single_axle_tonperhv_negative_direciton,
        worst_steering_double_axle_cnt_negative_direciton,
        worst_steering_double_axle_olhv_perc_negative_direciton,
        worst_steering_double_axle_tonperhv_negative_direciton,
        worst_non_steering_single_axle_cnt_negative_direciton,
        worst_non_steering_single_axle_olhv_perc_negative_direciton,
        worst_non_steering_single_axle_tonperhv_negative_direciton,
        worst_non_steering_double_axle_cnt_negative_direciton,
        worst_non_steering_double_axle_olhv_perc_negative_direciton,
        worst_non_steering_double_axle_tonperhv_negative_direciton,
        worst_triple_axle_cnt_negative_direciton,
        worst_triple_axle_olhv_perc_negative_direciton,
        worst_triple_axle_tonperhv_negative_direciton,
        bridge_formula_cnt_negative_direciton,
        bridge_formula_olhv_perc_negative_direciton,
        bridge_formula_tonperhv_negative_direciton,
        gross_formula_cnt_negative_direciton,
        gross_formula_olhv_perc_negative_direciton,
        gross_formula_tonperhv_negative_direciton,
        total_avg_cnt_negative_direciton,
        total_avg_olhv_perc_negative_direciton,
        total_avg_tonperhv_negative_direciton
        )
    values(
        '{header_id}',
        {egrl_percent},
        {egrl_percent_positive_direction},
        {egrl_percent_negative_direction},
        {egrw_percent},
        {egrw_percent_positive_direction},
        {egrw_percent_negative_direction},
        {num_weighed},
        {num_weighed_positive_direction},
        {num_weighed_negative_direction},
        {mean_equivalent_axle_mass},
        {mean_equivalent_axle_mass_positive_direction},
        {mean_equivalent_axle_mass_negative_direction},
        {mean_axle_spacing},
        {mean_axle_spacing_positive_direction},
        {mean_axle_spacing_negative_direction},
        {e80_per_axle},
        {e80_per_axle_positive_direction},
        {e80_per_axle_negative_direction},
        {olhv},
        {olhv_positive_direction},
        {olhv_negative_direction},
        {olhv_percent},
        {olhv_percent_positive_direction},
        {olhv_percent_negative_direction},
        {tonnage_generated},
        {tonnage_generated_positive_direction},
        {tonnage_generated_negative_direction},
        {olton},
        {olton_positive_direction},
        {olton_negative_direction},
        {olton_percent},
        {olton_percent_positive_direction},
        {olton_percent_negative_direction},
        {ole80},
        {ole80_positive_direction},
        {ole80_negative_direction},
        {ole80_percent},
        {ole80_percent_positive_direction},
        {ole80_percent_negative_direction},
        {xe80},
        {xe80_positive_direction},
        {xe80_negative_direction},
        {xe80_percent},
        {xe80_percent_positive_direction},
        {xe80_percent_negative_direction},
        {e80_per_day},
        {e80_per_day_positive_direction},
        {e80_per_day_negative_direction},
        {e80_per_heavy_vehicle},
        {e80_per_heavy_vehicle_positive_direction},
        {e80_per_heavy_vehicle_negative_direction},
        {worst_steering_single_axle_cnt},
        {worst_steering_single_axle_olhv_perc},
        {worst_steering_single_axle_tonperhv},
        {worst_steering_double_axle_cnt},
        {worst_steering_double_axle_olhv_perc},
        {worst_steering_double_axle_tonperhv},
        {worst_non_steering_single_axle_cnt},
        {worst_non_steering_single_axle_olhv_perc},
        {worst_non_steering_single_axle_tonperhv},
        {worst_non_steering_double_axle_cnt},
        {worst_non_steering_double_axle_olhv_perc},
        {worst_non_steering_double_axle_tonperhv},
        {worst_triple_axle_cnt},
        {worst_triple_axle_olhv_perc},
        {worst_triple_axle_tonperhv},
        {bridge_formula_cnt},
        {bridge_formula_olhv_perc},
        {bridge_formula_tonperhv},
        {gross_formula_cnt},
        {gross_formula_olhv_perc},
        {gross_formula_tonperhv},
        {total_avg_cnt},
        {total_avg_olhv_perc},
        {total_avg_tonperhv},
        {worst_steering_single_axle_cnt_positive_direciton},
        {worst_steering_single_axle_olhv_perc_positive_direciton},
        {worst_steering_single_axle_tonperhv_positive_direciton},
        {worst_steering_double_axle_cnt_positive_direciton},
        {worst_steering_double_axle_olhv_perc_positive_direciton},
        {worst_steering_double_axle_tonperhv_positive_direciton},
        {worst_non_steering_single_axle_cnt_positive_direciton},
        {worst_non_steering_single_axle_olhv_perc_positive_direciton},
        {worst_non_steering_single_axle_tonperhv_positive_direciton},
        {worst_non_steering_double_axle_cnt_positive_direciton},
        {worst_non_steering_double_axle_olhv_perc_positive_direciton},
        {worst_non_steering_double_axle_tonperhv_positive_direciton},
        {worst_triple_axle_cnt_positive_direciton},
        {worst_triple_axle_olhv_perc_positive_direciton},
        {worst_triple_axle_tonperhv_positive_direciton},
        {bridge_formula_cnt_positive_direciton},
        {bridge_formula_olhv_perc_positive_direciton},
        {bridge_formula_tonperhv_positive_direciton},
        {gross_formula_cnt_positive_direciton},
        {gross_formula_olhv_perc_positive_direciton},
        {gross_formula_tonperhv_positive_direciton},
        {total_avg_cnt_positive_direciton},
        {total_avg_olhv_perc_positive_direciton},
        {total_avg_tonperhv_positive_direciton},
        {worst_steering_single_axle_cnt_negative_direciton},
        {worst_steering_single_axle_olhv_perc_negative_direciton},
        {worst_steering_single_axle_tonperhv_negative_direciton},
        {worst_steering_double_axle_cnt_negative_direciton},
        {worst_steering_double_axle_olhv_perc_negative_direciton},
        {worst_steering_double_axle_tonperhv_negative_direciton},
        {worst_non_steering_single_axle_cnt_negative_direciton},
        {worst_non_steering_single_axle_olhv_perc_negative_direciton},
        {worst_non_steering_single_axle_tonperhv_negative_direciton},
        {worst_non_steering_double_axle_cnt_negative_direciton},
        {worst_non_steering_double_axle_olhv_perc_negative_direciton},
        {worst_non_steering_double_axle_tonperhv_negative_direciton},
        {worst_triple_axle_cnt_negative_direciton},
        {worst_triple_axle_olhv_perc_negative_direciton},
        {worst_triple_axle_tonperhv_negative_direciton},
        {bridge_formula_cnt_negative_direciton},
        {bridge_formula_olhv_perc_negative_direciton},
        {bridge_formula_tonperhv_negative_direciton},
        {gross_formula_cnt_negative_direciton},
        {gross_formula_olhv_perc_negative_direciton},
        {gross_formula_tonperhv_negative_direciton},
        {total_avg_cnt_negative_direciton},
        {total_avg_olhv_perc_negative_direciton},
        {total_avg_tonperhv_negative_direciton}
        ) ON CONFLICT ON CONSTRAINT electronic_count_header_hswim_pkey DO UPDATE SET
            egrl_percent = coalesce(excluded.egrl_percent,electronic_count_header_hswim.egrl_percent),
            egrl_percent_positive_direction = coalesce(excluded.egrl_percent_positive_direction,electronic_count_header_hswim.egrl_percent_positive_direction),
            egrl_percent_negative_direction = coalesce(excluded.egrl_percent_negative_direction,electronic_count_header_hswim.egrl_percent_negative_direction),
            egrw_percent = coalesce(excluded.egrw_percent,electronic_count_header_hswim.egrw_percent),
            egrw_percent_positive_direction = coalesce(excluded.egrw_percent_positive_direction,electronic_count_header_hswim.egrw_percent_positive_direction),
            egrw_percent_negative_direction = coalesce(excluded.egrw_percent_negative_direction,electronic_count_header_hswim.egrw_percent_negative_direction),
            num_weighed = coalesce(excluded.num_weighed,electronic_count_header_hswim.num_weighed),
            num_weighed_positive_direction = coalesce(excluded.num_weighed_positive_direction,electronic_count_header_hswim.num_weighed_positive_direction),
            num_weighed_negative_direction = coalesce(excluded.num_weighed_negative_direction,electronic_count_header_hswim.num_weighed_negative_direction),
            mean_equivalent_axle_mass = coalesce(excluded.mean_equivalent_axle_mass,electronic_count_header_hswim.mean_equivalent_axle_mass),
            mean_equivalent_axle_mass_positive_direction = coalesce(excluded.mean_equivalent_axle_mass_positive_direction,electronic_count_header_hswim.mean_equivalent_axle_mass_positive_direction),
            mean_equivalent_axle_mass_negative_direction = coalesce(excluded.mean_equivalent_axle_mass_negative_direction,electronic_count_header_hswim.mean_equivalent_axle_mass_negative_direction),
            mean_axle_spacing = coalesce(excluded.mean_axle_spacing,electronic_count_header_hswim.mean_axle_spacing),
            mean_axle_spacing_positive_direction = coalesce(excluded.mean_axle_spacing_positive_direction,electronic_count_header_hswim.mean_axle_spacing_positive_direction),
            mean_axle_spacing_negative_direction = coalesce(excluded.mean_axle_spacing_negative_direction,electronic_count_header_hswim.mean_axle_spacing_negative_direction),
            e80_per_axle = coalesce(excluded.e80_per_axle,electronic_count_header_hswim.e80_per_axle),
            e80_per_axle_positive_direction = coalesce(excluded.e80_per_axle_positive_direction,electronic_count_header_hswim.e80_per_axle_positive_direction),
            e80_per_axle_negative_direction = coalesce(excluded.e80_per_axle_negative_direction,electronic_count_header_hswim.e80_per_axle_negative_direction),
            olhv = coalesce(excluded.olhv,electronic_count_header_hswim.olhv),
            olhv_positive_direction = coalesce(excluded.olhv_positive_direction,electronic_count_header_hswim.olhv_positive_direction),
            olhv_negative_direction = coalesce(excluded.olhv_negative_direction,electronic_count_header_hswim.olhv_negative_direction),
            olhv_percent = coalesce(excluded.olhv_percent,electronic_count_header_hswim.olhv_percent),
            olhv_percent_positive_direction = coalesce(excluded.olhv_percent_positive_direction,electronic_count_header_hswim.olhv_percent_positive_direction),
            olhv_percent_negative_direction = coalesce(excluded.olhv_percent_negative_direction,electronic_count_header_hswim.olhv_percent_negative_direction),
            tonnage_generated = coalesce(excluded.tonnage_generated,electronic_count_header_hswim.tonnage_generated),
            tonnage_generated_positive_direction = coalesce(excluded.tonnage_generated_positive_direction,electronic_count_header_hswim.tonnage_generated_positive_direction),
            tonnage_generated_negative_direction = coalesce(excluded.tonnage_generated_negative_direction,electronic_count_header_hswim.tonnage_generated_negative_direction),
            olton = coalesce(excluded.olton,electronic_count_header_hswim.olton),
            olton_positive_direction = coalesce(excluded.olton_positive_direction,electronic_count_header_hswim.olton_positive_direction),
            olton_negative_direction = coalesce(excluded.olton_negative_direction,electronic_count_header_hswim.olton_negative_direction),
            olton_percent = coalesce(excluded.olton_percent,electronic_count_header_hswim.olton_percent),
            olton_percent_positive_direction = coalesce(excluded.olton_percent_positive_direction,electronic_count_header_hswim.olton_percent_positive_direction),
            olton_percent_negative_direction = coalesce(excluded.olton_percent_negative_direction,electronic_count_header_hswim.olton_percent_negative_direction),
            ole80 = coalesce(excluded.ole80,electronic_count_header_hswim.ole80),
            ole80_positive_direction = coalesce(excluded.ole80_positive_direction,electronic_count_header_hswim.ole80_positive_direction),
            ole80_negative_direction = coalesce(excluded.ole80_negative_direction,electronic_count_header_hswim.ole80_negative_direction),
            ole80_percent = coalesce(excluded.ole80_percent,electronic_count_header_hswim.ole80_percent),
            ole80_percent_positive_direction = coalesce(excluded.ole80_percent_positive_direction,electronic_count_header_hswim.ole80_percent_positive_direction),
            ole80_percent_negative_direction = coalesce(excluded.ole80_percent_negative_direction,electronic_count_header_hswim.ole80_percent_negative_direction),
            xe80 = coalesce(excluded.xe80,electronic_count_header_hswim.xe80),
            xe80_positive_direction = coalesce(excluded.xe80_positive_direction,electronic_count_header_hswim.xe80_positive_direction),
            xe80_negative_direction = coalesce(excluded.xe80_negative_direction,electronic_count_header_hswim.xe80_negative_direction),
            xe80_percent = coalesce(excluded.xe80_percent,electronic_count_header_hswim.xe80_percent),
            xe80_percent_positive_direction = coalesce(excluded.xe80_percent_positive_direction,electronic_count_header_hswim.xe80_percent_positive_direction),
            xe80_percent_negative_direction = coalesce(excluded.xe80_percent_negative_direction,electronic_count_header_hswim.xe80_percent_negative_direction),
            e80_per_day = coalesce(excluded.e80_per_day,electronic_count_header_hswim.e80_per_day),
            e80_per_day_positive_direction = coalesce(excluded.e80_per_day_positive_direction,electronic_count_header_hswim.e80_per_day_positive_direction),
            e80_per_day_negative_direction = coalesce(excluded.e80_per_day_negative_direction,electronic_count_header_hswim.e80_per_day_negative_direction),
            e80_per_heavy_vehicle = coalesce(excluded.e80_per_heavy_vehicle,electronic_count_header_hswim.e80_per_heavy_vehicle),
            e80_per_heavy_vehicle_positive_direction = coalesce(excluded.e80_per_heavy_vehicle_positive_direction,electronic_count_header_hswim.e80_per_heavy_vehicle_positive_direction),
            e80_per_heavy_vehicle_negative_direction = coalesce(excluded.e80_per_heavy_vehicle_negative_direction,electronic_count_header_hswim.e80_per_heavy_vehicle_negative_direction),
            worst_steering_single_axle_cnt = coalesce(excluded.worst_steering_single_axle_cnt,electronic_count_header_hswim.worst_steering_single_axle_cnt),
            worst_steering_single_axle_olhv_perc = coalesce(excluded.worst_steering_single_axle_olhv_perc,electronic_count_header_hswim.worst_steering_single_axle_olhv_perc),
            worst_steering_single_axle_tonperhv = coalesce(excluded.worst_steering_single_axle_tonperhv,electronic_count_header_hswim.worst_steering_single_axle_tonperhv),
            worst_steering_double_axle_cnt = coalesce(excluded.worst_steering_double_axle_cnt,electronic_count_header_hswim.worst_steering_double_axle_cnt),
            worst_steering_double_axle_olhv_perc = coalesce(excluded.worst_steering_double_axle_olhv_perc,electronic_count_header_hswim.worst_steering_double_axle_olhv_perc),
            worst_steering_double_axle_tonperhv = coalesce(excluded.worst_steering_double_axle_tonperhv,electronic_count_header_hswim.worst_steering_double_axle_tonperhv),
            worst_non_steering_single_axle_cnt = coalesce(excluded.worst_non_steering_single_axle_cnt,electronic_count_header_hswim.worst_non_steering_single_axle_cnt),
            worst_non_steering_single_axle_olhv_perc = coalesce(excluded.worst_non_steering_single_axle_olhv_perc,electronic_count_header_hswim.worst_non_steering_single_axle_olhv_perc),
            worst_non_steering_single_axle_tonperhv = coalesce(excluded.worst_non_steering_single_axle_tonperhv,electronic_count_header_hswim.worst_non_steering_single_axle_tonperhv),
            worst_non_steering_double_axle_cnt = coalesce(excluded.worst_non_steering_double_axle_cnt,electronic_count_header_hswim.worst_non_steering_double_axle_cnt),
            worst_non_steering_double_axle_olhv_perc = coalesce(excluded.worst_non_steering_double_axle_olhv_perc,electronic_count_header_hswim.worst_non_steering_double_axle_olhv_perc),
            worst_non_steering_double_axle_tonperhv = coalesce(excluded.worst_non_steering_double_axle_tonperhv,electronic_count_header_hswim.worst_non_steering_double_axle_tonperhv),
            worst_triple_axle_cnt = coalesce(excluded.worst_triple_axle_cnt,electronic_count_header_hswim.worst_triple_axle_cnt),
            worst_triple_axle_olhv_perc = coalesce(excluded.worst_triple_axle_olhv_perc,electronic_count_header_hswim.worst_triple_axle_olhv_perc),
            worst_triple_axle_tonperhv = coalesce(excluded.worst_triple_axle_tonperhv,electronic_count_header_hswim.worst_triple_axle_tonperhv),
            bridge_formula_cnt = coalesce(excluded.bridge_formula_cnt,electronic_count_header_hswim.bridge_formula_cnt),
            bridge_formula_olhv_perc = coalesce(excluded.bridge_formula_olhv_perc,electronic_count_header_hswim.bridge_formula_olhv_perc),
            bridge_formula_tonperhv = coalesce(excluded.bridge_formula_tonperhv,electronic_count_header_hswim.bridge_formula_tonperhv),
            gross_formula_cnt = coalesce(excluded.gross_formula_cnt,electronic_count_header_hswim.gross_formula_cnt),
            gross_formula_olhv_perc = coalesce(excluded.gross_formula_olhv_perc,electronic_count_header_hswim.gross_formula_olhv_perc),
            gross_formula_tonperhv = coalesce(excluded.gross_formula_tonperhv,electronic_count_header_hswim.gross_formula_tonperhv),
            total_avg_cnt = coalesce(excluded.total_avg_cnt,electronic_count_header_hswim.total_avg_cnt),
            total_avg_olhv_perc = coalesce(excluded.total_avg_olhv_perc,electronic_count_header_hswim.total_avg_olhv_perc),
            total_avg_tonperhv = coalesce(excluded.total_avg_tonperhv,electronic_count_header_hswim.total_avg_tonperhv),
            worst_steering_single_axle_cnt_positive_direciton = coalesce(excluded.worst_steering_single_axle_cnt_positive_direciton,electronic_count_header_hswim.worst_steering_single_axle_cnt_positive_direciton),
            worst_steering_single_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_steering_single_axle_olhv_perc_positive_direciton,electronic_count_header_hswim.worst_steering_single_axle_olhv_perc_positive_direciton),
            worst_steering_single_axle_tonperhv_positive_direciton = coalesce(excluded.worst_steering_single_axle_tonperhv_positive_direciton,electronic_count_header_hswim.worst_steering_single_axle_tonperhv_positive_direciton),
            worst_steering_double_axle_cnt_positive_direciton = coalesce(excluded.worst_steering_double_axle_cnt_positive_direciton,electronic_count_header_hswim.worst_steering_double_axle_cnt_positive_direciton),
            worst_steering_double_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_steering_double_axle_olhv_perc_positive_direciton,electronic_count_header_hswim.worst_steering_double_axle_olhv_perc_positive_direciton),
            worst_steering_double_axle_tonperhv_positive_direciton = coalesce(excluded.worst_steering_double_axle_tonperhv_positive_direciton,electronic_count_header_hswim.worst_steering_double_axle_tonperhv_positive_direciton),
            worst_non_steering_single_axle_cnt_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_cnt_positive_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_cnt_positive_direciton),
            worst_non_steering_single_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_olhv_perc_positive_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_olhv_perc_positive_direciton),
            worst_non_steering_single_axle_tonperhv_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_tonperhv_positive_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_tonperhv_positive_direciton),
            worst_non_steering_double_axle_cnt_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_cnt_positive_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_cnt_positive_direciton),
            worst_non_steering_double_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_olhv_perc_positive_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_olhv_perc_positive_direciton),
            worst_non_steering_double_axle_tonperhv_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_tonperhv_positive_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_tonperhv_positive_direciton),
            worst_triple_axle_cnt_positive_direciton = coalesce(excluded.worst_triple_axle_cnt_positive_direciton,electronic_count_header_hswim.worst_triple_axle_cnt_positive_direciton),
            worst_triple_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_triple_axle_olhv_perc_positive_direciton,electronic_count_header_hswim.worst_triple_axle_olhv_perc_positive_direciton),
            worst_triple_axle_tonperhv_positive_direciton = coalesce(excluded.worst_triple_axle_tonperhv_positive_direciton,electronic_count_header_hswim.worst_triple_axle_tonperhv_positive_direciton),
            bridge_formula_cnt_positive_direciton = coalesce(excluded.bridge_formula_cnt_positive_direciton,electronic_count_header_hswim.bridge_formula_cnt_positive_direciton),
            bridge_formula_olhv_perc_positive_direciton = coalesce(excluded.bridge_formula_olhv_perc_positive_direciton,electronic_count_header_hswim.bridge_formula_olhv_perc_positive_direciton),
            bridge_formula_tonperhv_positive_direciton = coalesce(excluded.bridge_formula_tonperhv_positive_direciton,electronic_count_header_hswim.bridge_formula_tonperhv_positive_direciton),
            gross_formula_cnt_positive_direciton = coalesce(excluded.gross_formula_cnt_positive_direciton,electronic_count_header_hswim.gross_formula_cnt_positive_direciton),
            gross_formula_olhv_perc_positive_direciton = coalesce(excluded.gross_formula_olhv_perc_positive_direciton,electronic_count_header_hswim.gross_formula_olhv_perc_positive_direciton),
            gross_formula_tonperhv_positive_direciton = coalesce(excluded.gross_formula_tonperhv_positive_direciton,electronic_count_header_hswim.gross_formula_tonperhv_positive_direciton),
            total_avg_cnt_positive_direciton = coalesce(excluded.total_avg_cnt_positive_direciton,electronic_count_header_hswim.total_avg_cnt_positive_direciton),
            total_avg_olhv_perc_positive_direciton = coalesce(excluded.total_avg_olhv_perc_positive_direciton,electronic_count_header_hswim.total_avg_olhv_perc_positive_direciton),
            total_avg_tonperhv_positive_direciton = coalesce(excluded.total_avg_tonperhv_positive_direciton,electronic_count_header_hswim.total_avg_tonperhv_positive_direciton),
            worst_steering_single_axle_cnt_negative_direciton = coalesce(excluded.worst_steering_single_axle_cnt_negative_direciton,electronic_count_header_hswim.worst_steering_single_axle_cnt_negative_direciton),
            worst_steering_single_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_steering_single_axle_olhv_perc_negative_direciton,electronic_count_header_hswim.worst_steering_single_axle_olhv_perc_negative_direciton),
            worst_steering_single_axle_tonperhv_negative_direciton = coalesce(excluded.worst_steering_single_axle_tonperhv_negative_direciton,electronic_count_header_hswim.worst_steering_single_axle_tonperhv_negative_direciton),
            worst_steering_double_axle_cnt_negative_direciton = coalesce(excluded.worst_steering_double_axle_cnt_negative_direciton,electronic_count_header_hswim.worst_steering_double_axle_cnt_negative_direciton),
            worst_steering_double_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_steering_double_axle_olhv_perc_negative_direciton,electronic_count_header_hswim.worst_steering_double_axle_olhv_perc_negative_direciton),
            worst_steering_double_axle_tonperhv_negative_direciton = coalesce(excluded.worst_steering_double_axle_tonperhv_negative_direciton,electronic_count_header_hswim.worst_steering_double_axle_tonperhv_negative_direciton),
            worst_non_steering_single_axle_cnt_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_cnt_negative_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_cnt_negative_direciton),
            worst_non_steering_single_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_olhv_perc_negative_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_olhv_perc_negative_direciton),
            worst_non_steering_single_axle_tonperhv_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_tonperhv_negative_direciton,electronic_count_header_hswim.worst_non_steering_single_axle_tonperhv_negative_direciton),
            worst_non_steering_double_axle_cnt_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_cnt_negative_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_cnt_negative_direciton),
            worst_non_steering_double_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_olhv_perc_negative_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_olhv_perc_negative_direciton),
            worst_non_steering_double_axle_tonperhv_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_tonperhv_negative_direciton,electronic_count_header_hswim.worst_non_steering_double_axle_tonperhv_negative_direciton),
            worst_triple_axle_cnt_negative_direciton = coalesce(excluded.worst_triple_axle_cnt_negative_direciton,electronic_count_header_hswim.worst_triple_axle_cnt_negative_direciton),
            worst_triple_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_triple_axle_olhv_perc_negative_direciton,electronic_count_header_hswim.worst_triple_axle_olhv_perc_negative_direciton),
            worst_triple_axle_tonperhv_negative_direciton = coalesce(excluded.worst_triple_axle_tonperhv_negative_direciton,electronic_count_header_hswim.worst_triple_axle_tonperhv_negative_direciton),
            bridge_formula_cnt_negative_direciton = coalesce(excluded.bridge_formula_cnt_negative_direciton,electronic_count_header_hswim.bridge_formula_cnt_negative_direciton),
            bridge_formula_olhv_perc_negative_direciton = coalesce(excluded.bridge_formula_olhv_perc_negative_direciton,electronic_count_header_hswim.bridge_formula_olhv_perc_negative_direciton),
            bridge_formula_tonperhv_negative_direciton = coalesce(excluded.bridge_formula_tonperhv_negative_direciton,electronic_count_header_hswim.bridge_formula_tonperhv_negative_direciton),
            gross_formula_cnt_negative_direciton = coalesce(excluded.gross_formula_cnt_negative_direciton,electronic_count_header_hswim.gross_formula_cnt_negative_direciton),
            gross_formula_olhv_perc_negative_direciton = coalesce(excluded.gross_formula_olhv_perc_negative_direciton,electronic_count_header_hswim.gross_formula_olhv_perc_negative_direciton),
            gross_formula_tonperhv_negative_direciton = coalesce(excluded.gross_formula_tonperhv_negative_direciton,electronic_count_header_hswim.gross_formula_tonperhv_negative_direciton),
            total_avg_cnt_negative_direciton = coalesce(excluded.total_avg_cnt_negative_direciton,electronic_count_header_hswim.total_avg_cnt_negative_direciton),
            total_avg_olhv_perc_negative_direciton = coalesce(excluded.total_avg_olhv_perc_negative_direciton,electronic_count_header_hswim.total_avg_olhv_perc_negative_direciton),
            total_avg_tonperhv_negative_direciton = coalesce(excluded.total_avg_tonperhv_negative_direciton,electronic_count_header_hswim.total_avg_tonperhv_negative_direciton)
        ;
    """
    return INSERT_STRING

In [10]:
def wim_stations_header_update(header_id):
    SELECT_TYPE10_QRY = f"""SELECT 
        *
        FROM trafc.electronic_count_data_type_10 t10
        left join traf_lu.vehicle_classes_scheme_08 c on c.id = t10.vehicle_class_code_primary_scheme
        where t10.header_id = '{header_id}'
        """
    AXLE_SPACING_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        axs.axle_spacing_number,
        axs.axle_spacing_cm,
        FROM trafc.electronic_count_data_type_10 t10
        left join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
        where t10.header_id = '{header_id}'
        """
    WHEEL_MASS_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        wm.wheel_mass_number,
        wm.wheel_mass,
        vm.kg as vehicle_mass_limit_kg,
        sum(wm.wheel_mass)*2 over(partition by t10.id) as gross_mass
        FROM trafc.electronic_count_data_type_10 t10
        left join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
        left join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
        where t10.header_id = '{header_id}'
        """

    df = pd.read_sql_query(SELECT_TYPE10_QRY,config.ENGINE)
    df2 = pd.read_sql_query(AXLE_SPACING_SELECT_QRY,config.ENGINE)
    df3 = pd.read_sql_query(WHEEL_MASS_SELECT_QRY,config.ENGINE)

    UPDATE_STRING = f"""
    update
        trafc.electronic_count_header set
        egrl_percent {(df.loc[data['edit_code']==2].count()[0]/df.count()[0])*100},
        egrw_percent {},
        mean_equivalent_axle_mass = {(df3.groupby(pd.Grouper(key='id')).mean()*2).mean().round(2)},
        mean_equivalent_axle_mass_positive_direction = {(df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id')).mean()*2).mean().round(2)},
        mean_equivalent_axle_mass_negative_direction = {(df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id')).mean()*2).mean().round(2)},
        mean_axle_spacing = {(df3.groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round()},
        mean_axle_spacing_positive_direction = {(df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round()},
        mean_axle_spacing_negative_direction = {(df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round()},
        e80_per_axle = {((df3['wheel_mass']*2/8200)**4.2).sum().round()},
        e80_per_axle_positive_direction = {((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round()},
        e80_per_axle_negative_direction = {((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round()},
        olhv = {len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique())},
        olhv_positive_direction = {len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique())},
        olhv_negative_direction = {len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique())},
        olhv_percent = {round(((len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique())/len(df3['id'].unique()))*100),2)},
        olhv_percent_positive_direction = {round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique())/len(df3.loc[df3['direction']=='P']['id'].unique()))*100),2)},
        olhv_percent_negative_direction = {round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique())/len(df3.loc[df3['direction']=='N']['id'].unique()))*100),2)},
        tonnage_generated = {((df3['wheel_mass']*2).sum()/1000).round().astype(int)},
        tonnage_generated_positive_direction = {((df3.loc[df3['direction']=='P']['wheel_mass']*2).sum()/1000).round().astype(int)},
        tonnage_generated_negative_direction = {((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000).round().astype(int)},
        olton = {(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum().astype(int)},
        olton_positive_direction = {(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum().astype(int)},
        olton_negative_direction = {(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum().astype(int)},
        olton_percent = {round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum()/(df3['wheel_mass']*2).sum())*100,2)},
        olton_percent_positive_direction = {round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum()/(df3.loc[df3['direction']=='P']['wheel_mass']*2).sum())*100,2)},
        olton_percent_negative_direction = {round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum()/(df3.loc[df3['direction']=='N']['wheel_mass']*2).sum())*100,2)},
        ole80 = {((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum().round()},
        ole80_positive_direction = {((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()},
        ole80_negative_direction = {((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()},
        ole80_percent = {((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum().round()/((df3['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2)},
        ole80_percent_positive_direction = {((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2)},
        ole80_percent_negative_direction = {((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2)},
        xe80 = {},
        xe80_positive_direction = {},
        xe80_negative_direction = {},
        xe80_percent = {},
        xe80_percent_positive_direction = {},
        xe80_percent_negative_direction = {},
        e80_per_day = {((((df3['wheel_mass']*2/8200)**4.2).sum().round()/df3.groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)},
        e80_per_day_positive_direction = {((((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)},
        e80_per_day_negative_direction = {((((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2)},
        e80_per_heavy_vehicle = {((((df3.loc[df3['vehicle_class_code_primary_scheme']>3]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['vehicle_class_code_primary_scheme']>3].count()[0])*100).round(2)},
        e80_per_heavy_vehicle_positive_direction = {((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')].count()[0])*100).round(2)},
        e80_per_heavy_vehicle_negative_direction = {((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='N')].count()[0])*100).round(2)},
        worst_steering_single_axle_cnt = {},
        worst_steering_single_axle_olhv_perc = {},
        worst_steering_single_axle_tonperhv = {},
        worst_steering_double_axle_cnt = {},
        worst_steering_double_axle_olhv_perc = {},
        worst_steering_double_axle_tonperhv = {},
        worst_non_steering_single_axle_cnt = {},
        worst_non_steering_single_axle_olhv_perc = {},
        worst_non_steering_single_axle_tonperhv = {},
        worst_non_steering_double_axle_cnt = {},
        worst_non_steering_double_axle_olhv_perc = {},
        worst_non_steering_double_axle_tonperhv = {},
        worst_triple_axle_cnt = {},
        worst_triple_axle_olhv_perc = {},
        worst_triple_axle_tonperhv = {},
        bridge_formula_cnt = {(18000 + 2.1 * (df2.loc[df2['axle_spacing_number']>1].groupby('id')['axle_spacing_cm'].sum().mean().round(2))).round(2)},
        bridge_formula_olhv_perc = {},
        bridge_formula_tonperhv = {},
        gross_formula_cnt = {},
        gross_formula_olhv_perc = {},
        gross_formula_tonperhv = {},
        total_avg_cnt = {},
        total_avg_olhv_perc = {},
        total_avg_tonperhv = {},
        worst_steering_single_axle_cnt_positive_direciton = {},
        worst_steering_single_axle_olhv_perc_positive_direciton = {},
        worst_steering_single_axle_tonperhv_positive_direciton = {},
        worst_steering_double_axle_cnt_positive_direciton = {},
        worst_steering_double_axle_olhv_perc_positive_direciton = {},
        worst_steering_double_axle_tonperhv_positive_direciton = {},
        worst_non_steering_single_axle_cnt_positive_direciton = {},
        worst_non_steering_single_axle_olhv_perc_positive_direciton = {},
        worst_non_steering_single_axle_tonperhv_positive_direciton = {},
        worst_non_steering_double_axle_cnt_positive_direciton = {},
        worst_non_steering_double_axle_olhv_perc_positive_direciton = {},
        worst_non_steering_double_axle_tonperhv_positive_direciton = {},
        worst_triple_axle_cnt_positive_direciton = {},
        worst_triple_axle_olhv_perc_positive_direciton = {},
        worst_triple_axle_tonperhv_positive_direciton = {},
        bridge_formula_cnt_positive_direciton = {},
        bridge_formula_olhv_perc_positive_direciton = {},
        bridge_formula_tonperhv_positive_direciton = {},
        gross_formula_cnt_positive_direciton = {},
        gross_formula_olhv_perc_positive_direciton = {},
        gross_formula_tonperhv_positive_direciton = {},
        total_avg_cnt_positive_direciton = {},
        total_avg_olhv_perc_positive_direciton = {},
        total_avg_tonperhv_positive_direciton = {},
        worst_steering_single_axle_cnt_negative_direciton = {},
        worst_steering_single_axle_olhv_perc_negative_direciton = {},
        worst_steering_single_axle_tonperhv_negative_direciton = {},
        worst_steering_double_axle_cnt_negative_direciton = {},
        worst_steering_double_axle_olhv_perc_negative_direciton = {},
        worst_steering_double_axle_tonperhv_negative_direciton = {},
        worst_non_steering_single_axle_cnt_negative_direciton = {},
        worst_non_steering_single_axle_olhv_perc_negative_direciton = {},
        worst_non_steering_single_axle_tonperhv_negative_direciton = {},
        worst_non_steering_double_axle_cnt_negative_direciton = {},
        worst_non_steering_double_axle_olhv_perc_negative_direciton = {},
        worst_non_steering_double_axle_tonperhv_negative_direciton = {},
        worst_triple_axle_cnt_negative_direciton = {},
        worst_triple_axle_olhv_perc_negative_direciton = {},
        worst_triple_axle_tonperhv_negative_direciton = {},
        bridge_formula_cnt_negative_direciton = {},
        bridge_formula_olhv_perc_negative_direciton = {},
        bridge_formula_tonperhv_negative_direciton = {},
        gross_formula_cnt_negative_direciton = {},
        gross_formula_olhv_perc_negative_direciton = {},
        gross_formula_tonperhv_negative_direciton = {},
        total_avg_cnt_negative_direciton = {},
        total_avg_olhv_perc_negative_direciton = {},
        total_avg_tonperhv_negative_direciton = {}
    where
        header_id = '{header_id}';
    """

    return UPDATE_STRING

SyntaxError: f-string: empty expression not allowed (2284184335.py, line 167)

In [11]:
GVML = pd.read_sql_query(queries.GET_GROSS_VEHICLE_MASS_LIMITS, config.ENGINE)
AXLL = pd.read_sql_query(queries.GET_AXLE_LOAD_LIMITS, config.ENGINE)
header_ids = pd.read_sql_query(GET_HSWIM_HEADER_IDS, config.ENGINE)
for header_id in list(header_ids['header_id'].astype(str)):
    SELECT_TYPE10_QRY = f"""SELECT * FROM trafc.electronic_count_data_type_10 t10
        left join traf_lu.vehicle_classes_scheme_08 c on c.id = t10.vehicle_class_code_primary_scheme
        where t10.header_id = '{header_id}'
        """
    AXLE_SPACING_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        axs.axle_spacing_number,
        axs.axle_spacing_cm
        FROM trafc.electronic_count_data_type_10 t10
        left join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
        where t10.header_id = '{header_id}'
        """
    WHEEL_MASS_SELECT_QRY = f"""SELECT 
        t10.id,
        t10.header_id, 
        t10.start_datetime,
        t10.edit_code,
        t10.vehicle_class_code_primary_scheme, 
        t10.vehicle_class_code_secondary_scheme,
        t10.direction,
        t10.axle_count,
        wm.wheel_mass_number,
        wm.wheel_mass,
        vm.kg as vehicle_mass_limit_kg,
        sum(wm.wheel_mass*2) over(partition by t10.id) as gross_mass
        FROM trafc.electronic_count_data_type_10 t10
        left join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
        left join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
        where t10.header_id = '{header_id}'
        """
    df = pd.read_sql_query(SELECT_TYPE10_QRY,config.ENGINE)
    df2 = pd.read_sql_query(AXLE_SPACING_SELECT_QRY,config.ENGINE)
    df3 = pd.read_sql_query(WHEEL_MASS_SELECT_QRY,config.ENGINE)

    # INSERT_STRING = f"""
    # insert into trafc.electronic_count_header_hswim (
    #     header_id,
    #     egrl_percent,
    #     egrl_percent_positive_direction,
    #     egrl_percent_negative_direction,
    #     egrw_percent ,
    #     egrw_percent_positive_direction ,
    #     egrw_percent_negative_direction ,
    #     num_weighed,
    #     num_weighed_positive_direction,
    #     num_weighed_negative_direction,
    #     mean_equivalent_axle_mass,
    #     mean_equivalent_axle_mass_positive_direction,
    #     mean_equivalent_axle_mass_negative_direction,
    #     mean_axle_spacing,
    #     mean_axle_spacing_positive_direction,
    #     mean_axle_spacing_negative_direction,
    #     e80_per_axle,
    #     e80_per_axle_positive_direction,
    #     e80_per_axle_negative_direction,
    #     olhv,
    #     olhv_positive_direction,
    #     olhv_negative_direction,
    #     olhv_percent,
    #     olhv_percent_positive_direction,
    #     olhv_percent_negative_direction,
    #     tonnage_generated,
    #     tonnage_generated_positive_direction,
    #     tonnage_generated_negative_direction,
    #     olton,
    #     olton_positive_direction,
    #     olton_negative_direction,
    #     olton_percent,
    #     olton_percent_positive_direction,
    #     olton_percent_negative_direction,
    #     ole80,
    #     ole80_positive_direction,
    #     ole80_negative_direction,
    #     ole80_percent,
    #     ole80_percent_positive_direction,
    #     ole80_percent_negative_direction,
    #     xe80,
    #     xe80_positive_direction,
    #     xe80_negative_direction,
    #     xe80_percent,
    #     xe80_percent_positive_direction,
    #     xe80_percent_negative_direction,
    #     e80_per_day,
    #     e80_per_day_positive_direction,
    #     e80_per_day_negative_direction,
    #     e80_per_heavy_vehicle,
    #     e80_per_heavy_vehicle_positive_direction,
    #     e80_per_heavy_vehicle_negative_direction,
    #     worst_steering_single_axle_cnt,
    #     worst_steering_single_axle_olhv_perc,
    #     worst_steering_single_axle_tonperhv,
    #     worst_steering_double_axle_cnt,
    #     worst_steering_double_axle_olhv_perc,
    #     worst_steering_double_axle_tonperhv,
    #     worst_non_steering_single_axle_cnt,
    #     worst_non_steering_single_axle_olhv_perc,
    #     worst_non_steering_single_axle_tonperhv,
    #     worst_non_steering_double_axle_cnt,
    #     worst_non_steering_double_axle_olhv_perc,
    #     worst_non_steering_double_axle_tonperhv,
    #     worst_triple_axle_cnt,
    #     worst_triple_axle_olhv_perc,
    #     worst_triple_axle_tonperhv,
    #     bridge_formula_cnt,
    #     bridge_formula_olhv_perc,
    #     bridge_formula_tonperhv,
    #     gross_formula_cnt,
    #     gross_formula_olhv_perc,
    #     gross_formula_tonperhv,
    #     total_avg_cnt,
    #     total_avg_olhv_perc,
    #     total_avg_tonperhv,
    #     worst_steering_single_axle_cnt_positive_direciton,
    #     worst_steering_single_axle_olhv_perc_positive_direciton,
    #     worst_steering_single_axle_tonperhv_positive_direciton,
    #     worst_steering_double_axle_cnt_positive_direciton,
    #     worst_steering_double_axle_olhv_perc_positive_direciton,
    #     worst_steering_double_axle_tonperhv_positive_direciton,
    #     worst_non_steering_single_axle_cnt_positive_direciton,
    #     worst_non_steering_single_axle_olhv_perc_positive_direciton,
    #     worst_non_steering_single_axle_tonperhv_positive_direciton,
    #     worst_non_steering_double_axle_cnt_positive_direciton,
    #     worst_non_steering_double_axle_olhv_perc_positive_direciton,
    #     worst_non_steering_double_axle_tonperhv_positive_direciton,
    #     worst_triple_axle_cnt_positive_direciton,
    #     worst_triple_axle_olhv_perc_positive_direciton,
    #     worst_triple_axle_tonperhv_positive_direciton,
    #     bridge_formula_cnt_positive_direciton,
    #     bridge_formula_olhv_perc_positive_direciton,
    #     bridge_formula_tonperhv_positive_direciton,
    #     gross_formula_cnt_positive_direciton,
    #     gross_formula_olhv_perc_positive_direciton,
    #     gross_formula_tonperhv_positive_direciton,
    #     total_avg_cnt_positive_direciton,
    #     total_avg_olhv_perc_positive_direciton,
    #     total_avg_tonperhv_positive_direciton,
    #     worst_steering_single_axle_cnt_negative_direciton,
    #     worst_steering_single_axle_olhv_perc_negative_direciton,
    #     worst_steering_single_axle_tonperhv_negative_direciton,
    #     worst_steering_double_axle_cnt_negative_direciton,
    #     worst_steering_double_axle_olhv_perc_negative_direciton,
    #     worst_steering_double_axle_tonperhv_negative_direciton,
    #     worst_non_steering_single_axle_cnt_negative_direciton,
    #     worst_non_steering_single_axle_olhv_perc_negative_direciton,
    #     worst_non_steering_single_axle_tonperhv_negative_direciton,
    #     worst_non_steering_double_axle_cnt_negative_direciton,
    #     worst_non_steering_double_axle_olhv_perc_negative_direciton,
    #     worst_non_steering_double_axle_tonperhv_negative_direciton,
    #     worst_triple_axle_cnt_negative_direciton,
    #     worst_triple_axle_olhv_perc_negative_direciton,
    #     worst_triple_axle_tonperhv_negative_direciton,
    #     bridge_formula_cnt_negative_direciton,
    #     bridge_formula_olhv_perc_negative_direciton,
    #     bridge_formula_tonperhv_negative_direciton,
    #     gross_formula_cnt_negative_direciton,
    #     gross_formula_olhv_perc_negative_direciton,
    #     gross_formula_tonperhv_negative_direciton,
    #     total_avg_cnt_negative_direciton,
    #     total_avg_olhv_perc_negative_direciton,
    #     total_avg_tonperhv_negative_direciton
    #     )
    # values(
        # '{header_id}',
    try:
        ((df.loc[df['edit_code']==2].count()[0]/df.count()[0])*100).round() or 0
    except:
        0
    try:
        ((df.loc[(df['edit_code']==2)&(df['direction']=='P')].count()[0]/df.loc[df['direction']=='P'].count()[0])*100).round() or 0
    except:
        0
    try:
        ((df.loc[(df['edit_code']==2)&(df['direction']=='P')].count()[0]/df.loc[df['direction']=='N'].count()[0])*100).round() or 0
    except:
        0
    try:
        (((df2.loc[df2['edit_code']==2].count()[0]+df3.loc[df3['edit_code']==2].count()[0])/df.count()[0])*100).round()  or 0
    except:
        0
    try:
        (((df2.loc[(df2['edit_code']==2)&(df2['direction']=='P')].count()[0]+df3.loc[(df3['edit_code']==2)&(df3['direction']=='P')].count()[0])/df.loc[df['direction']=='P'].count()[0])*100).round() or 0
    except:
        0
    try:
        (((df2.loc[(df2['edit_code']==2)&(df2['direction']=='N')].count()[0]+df3.loc[(df3['edit_code']==2)&(df3['direction']=='N')].count()[0])/df.loc[df['direction']=='N'].count()[0])*100).round() or 0
    except:
        0
    df3.groupby(pd.Grouper(key='id')).count().count()[0] or 0
    df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id')).count().count()[0] or 0
    df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id')).count().count()[0] or 0
    (df3.groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean().round(2) or 0
    (df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean().round(2) or 0
    (df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='id'))['wheel_mass'].mean()*2).mean().round(2) or 0
    (df2.groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round() or 0
    (df2.loc[df2['direction']=='P'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round() or 0
    (df2.loc[df2['direction']=='N'].groupby(pd.Grouper(key='id')).mean()).mean()['axle_spacing_number'].round() or 0
    (df3['wheel_mass']*2/8200)**4.2 or 0
    (df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2 or 0
    (df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2 or 0
    len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique()) or 0
    len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique()) or 0
    len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique()) or 0
    try:
        round(((len(df3.loc[df3['gross_mass']>df3['vehicle_mass_limit_kg']]['id'].unique())/len(df3['id'].unique()))*100),2) or 0
    except:
        0
    try:
        round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['id'].unique())/len(df3.loc[df3['direction']=='P']['id'].unique()))*100),2) or 0
    except:
        0
    try:
        round(((len(df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['id'].unique())/len(df3.loc[df3['direction']=='N']['id'].unique()))*100),2) or 0
    except:
        0
    ((df3['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    ((df3.loc[df3['direction']=='P']['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    ((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000).round().astype(int) or 0
    (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum().astype(int) or 0
    (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum().astype(int) or 0
    (df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum().astype(int) or 0
    try:
        round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2).sum()/(df3['wheel_mass']*2).sum())*100,2) or 0
    except:
        0
    try:
        round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2).sum()/(df3.loc[df3['direction']=='P']['wheel_mass']*2).sum())*100,2) or 0
    except:
        0
    try:
        round(((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2).sum()/(df3.loc[df3['direction']=='N']['wheel_mass']*2).sum())*100,2) or 0
    except:
        0
    ((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum().round() or 0
    ((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round() or 0
    ((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round() or 0
    try:
        ((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])]['wheel_mass']*2/8200)**4.2).sum().round()/((df3['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[(df3['gross_mass']>df3['vehicle_mass_limit_kg'])&(df3['direction']=='N')]['wheel_mass']*2/8200)**4.2).sum().round()/((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round())*100).round(2) or 0
    except:
        0
    ((df3['wheel_mass']*2/8200)**4.2)-(((df3['wheel_mass']*2*0.05)/8200)**4.2) or 0
    ((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2)-(((df3.loc[df3['direction']=='P']['wheel_mass']*2*0.05)/8200)**4.2) or 0
    ((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2)-(((df3.loc[df3['direction']=='N']['wheel_mass']*2*0.05)/8200)**4.2) or 0
    try:
        (((((df3['wheel_mass']*2/8200)**4.2)-(((df3['wheel_mass']*2*0.05)/8200)**4.2))/((df3['wheel_mass']*2/8200)**4.2))*100).round() or 0
    except:
        0
    try:
        (((((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2)-(((df3.loc[df3['direction']=='P']['wheel_mass']*2*0.05)/8200)**4.2))/((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2))*100).round() or 0
    except:
        0
    try:
        (((((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2)-(((df3.loc[df3['direction']=='N']['wheel_mass']*2*0.05)/8200)**4.2))/((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2))*100).round() or 0
    except:
        0
    try:
        ((((df3['wheel_mass']*2/8200)**4.2).sum().round()/df3.groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[df3['direction']=='P']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[df3['direction']=='N']['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().count()[0])*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[df3['vehicle_class_code_primary_scheme']>3]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[df3['vehicle_class_code_primary_scheme']>3].count()[0])*100).round(2) or 0
    except:
        0
    try:
        ((((df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')]['wheel_mass']*2/8200)**4.2).sum().round()/df3.loc[(df3['vehicle_class_code_primary_scheme']>3)&(df3['direction']=='P')].count()[0])*100).round(2) or 0
    except:
        0
    # ,    -- worst_steering_single_axle_cnt
    # ,    -- worst_steering_single_axle_olhv_perc
    # ,    -- worst_steering_single_axle_tonperhv
    # ,    -- worst_steering_double_axle_cnt
    # ,    -- worst_steering_double_axle_olhv_perc
    # ,    -- worst_steering_double_axle_tonperhv
    # ,    -- worst_non_steering_single_axle_cnt
    # ,    -- worst_non_steering_single_axle_olhv_perc
    # ,    -- worst_non_steering_single_axle_tonperhv
    # ,    -- worst_non_steering_double_axle_cnt
    # ,    -- worst_non_steering_double_axle_olhv_perc
    # ,    -- worst_non_steering_double_axle_tonperhv
    # ,    -- worst_triple_axle_cnt
    # ,    -- worst_triple_axle_olhv_perc
    # ,    -- worst_triple_axle_tonperhv
    (18000 + 2.1 * (df2.loc[df2['axle_spacing_number']>1].groupby('id')['axle_spacing_cm'].sum().mean().round(2))).round(2) or 0
    #     ,    -- bridge_formula_olhv_perc
    #     ,    -- bridge_formula_tonperhv
    #     ,    -- gross_formula_cnt
    #     ,    -- gross_formula_olhv_perc
    #     ,    -- gross_formula_tonperhv
    #     ,    -- total_avg_cnt
    #     ,    -- total_avg_olhv_perc
    #     ,    -- total_avg_tonperhv
    #     ,    -- worst_steering_single_axle_cnt_positive_direciton
    #     ,    -- worst_steering_single_axle_olhv_perc_positive_direciton
    #     ,    -- worst_steering_single_axle_tonperhv_positive_direciton
    #     ,    -- worst_steering_double_axle_cnt_positive_direciton
    #     ,    -- worst_steering_double_axle_olhv_perc_positive_direciton
    #     ,    -- worst_steering_double_axle_tonperhv_positive_direciton
    #     ,    -- worst_non_steering_single_axle_cnt_positive_direciton
    #     ,    -- worst_non_steering_single_axle_olhv_perc_positive_direciton
    #     ,    -- worst_non_steering_single_axle_tonperhv_positive_direciton
    #     ,    -- worst_non_steering_double_axle_cnt_positive_direciton
    #     ,    -- worst_non_steering_double_axle_olhv_perc_positive_direciton
    #     ,    -- worst_non_steering_double_axle_tonperhv_positive_direciton
    #     ,    -- worst_triple_axle_cnt_positive_direciton
    #     ,    -- worst_triple_axle_olhv_perc_positive_direciton
    #     ,    -- worst_triple_axle_tonperhv_positive_direciton
    #     ,    -- bridge_formula_cnt_positive_direciton
    #     ,    -- bridge_formula_olhv_perc_positive_direciton
    #     ,    -- bridge_formula_tonperhv_positive_direciton
    #     ,    -- gross_formula_cnt_positive_direciton
    #     ,    -- gross_formula_olhv_perc_positive_direciton
    #     ,    -- gross_formula_tonperhv_positive_direciton
    #     ,    -- total_avg_cnt_positive_direciton
    #     ,    -- total_avg_olhv_perc_positive_direciton
    #     ,    -- total_avg_tonperhv_positive_direciton
    #     ,    -- worst_steering_single_axle_cnt_negative_direciton
    #     ,    -- worst_steering_single_axle_olhv_perc_negative_direciton
    #     ,    -- worst_steering_single_axle_tonperhv_negative_direciton
    #     ,    -- worst_steering_double_axle_cnt_negative_direciton
    #     ,    -- worst_steering_double_axle_olhv_perc_negative_direciton
    #     ,    -- worst_steering_double_axle_tonperhv_negative_direciton
    #     ,    -- worst_non_steering_single_axle_cnt_negative_direciton
    #     ,    -- worst_non_steering_single_axle_olhv_perc_negative_direciton
    #     ,    -- worst_non_steering_single_axle_tonperhv_negative_direciton
    #     ,    -- worst_non_steering_double_axle_cnt_negative_direciton
    #     ,    -- worst_non_steering_double_axle_olhv_perc_negative_direciton
    #     ,    -- worst_non_steering_double_axle_tonperhv_negative_direciton
    #     ,    -- worst_triple_axle_cnt_negative_direciton
    #     ,    -- worst_triple_axle_olhv_perc_negative_direciton
    #     ,    -- worst_triple_axle_tonperhv_negative_direciton
    #     ,    -- bridge_formula_cnt_negative_direciton
    #     ,    -- bridge_formula_olhv_perc_negative_direciton
    #     ,    -- bridge_formula_tonperhv_negative_direciton
    #     ,    -- gross_formula_cnt_negative_direciton
    #     ,    -- gross_formula_olhv_perc_negative_direciton
    #     ,    -- gross_formula_tonperhv_negative_direciton
    #     ,    -- total_avg_cnt_negative_direciton
    #     ,    -- total_avg_olhv_perc_negative_direciton
    #         -- total_avg_tonperhv_negative_direciton
    #         ON CONFLICT ON CONSTRAINT electronic_count_header_hswim_pkey DO UPDATE SET
    #         egrl_percent = coalesce(excluded.egrl_percent,egrl_percent),
    #         egrl_percent_positive_direction = coalesce(excluded.egrl_percent_positive_direction,egrl_percent_positive_direction),
    #         egrl_percent_negative_direction = coalesce(excluded.egrl_percent_negative_direction,egrl_percent_negative_direction),
    #         egrw_percent = coalesce(excluded.egrw_percent,egrw_percent),
    #         egrw_percent_positive_direction = coalesce(excluded.egrw_percent_positive_direction,egrw_percent_positive_direction),
    #         egrw_percent_negative_direction = coalesce(excluded.egrw_percent_negative_direction,egrw_percent_negative_direction),
    #         num_weighed = coalesce(excluded.num_weighed,num_weighed),
    #         num_weighed_positive_direction = coalesce(excluded.num_weighed_positive_direction,num_weighed_positive_direction),
    #         num_weighed_negative_direction = coalesce(excluded.num_weighed_negative_direction,num_weighed_negative_direction),
    #         mean_equivalent_axle_mass = coalesce(excluded.mean_equivalent_axle_mass,mean_equivalent_axle_mass),
    #         mean_equivalent_axle_mass_positive_direction = coalesce(excluded.mean_equivalent_axle_mass_positive_direction,mean_equivalent_axle_mass_positive_direction),
    #         mean_equivalent_axle_mass_negative_direction = coalesce(excluded.mean_equivalent_axle_mass_negative_direction,mean_equivalent_axle_mass_negative_direction),
    #         mean_axle_spacing = coalesce(excluded.mean_axle_spacing,mean_axle_spacing),
    #         mean_axle_spacing_positive_direction = coalesce(excluded.mean_axle_spacing_positive_direction,mean_axle_spacing_positive_direction),
    #         mean_axle_spacing_negative_direction = coalesce(excluded.mean_axle_spacing_negative_direction,mean_axle_spacing_negative_direction),
    #         e80_per_axle = coalesce(excluded.e80_per_axle,e80_per_axle),
    #         e80_per_axle_positive_direction = coalesce(excluded.e80_per_axle_positive_direction,e80_per_axle_positive_direction),
    #         e80_per_axle_negative_direction = coalesce(excluded.e80_per_axle_negative_direction,e80_per_axle_negative_direction),
    #         olhv = coalesce(excluded.olhv,olhv),
    #         olhv_positive_direction = coalesce(excluded.olhv_positive_direction,olhv_positive_direction),
    #         olhv_negative_direction = coalesce(excluded.olhv_negative_direction,olhv_negative_direction),
    #         olhv_percent = coalesce(excluded.olhv_percent,olhv_percent),
    #         olhv_percent_positive_direction = coalesce(excluded.olhv_percent_positive_direction,olhv_percent_positive_direction),
    #         olhv_percent_negative_direction = coalesce(excluded.olhv_percent_negative_direction,olhv_percent_negative_direction),
    #         tonnage_generated = coalesce(excluded.tonnage_generated,tonnage_generated),
    #         tonnage_generated_positive_direction = coalesce(excluded.tonnage_generated_positive_direction,tonnage_generated_positive_direction),
    #         tonnage_generated_negative_direction = coalesce(excluded.tonnage_generated_negative_direction,tonnage_generated_negative_direction),
    #         olton = coalesce(excluded.olton,olton),
    #         olton_positive_direction = coalesce(excluded.olton_positive_direction,olton_positive_direction),
    #         olton_negative_direction = coalesce(excluded.olton_negative_direction,olton_negative_direction),
    #         olton_percent = coalesce(excluded.olton_percent,olton_percent),
    #         olton_percent_positive_direction = coalesce(excluded.olton_percent_positive_direction,olton_percent_positive_direction),
    #         olton_percent_negative_direction = coalesce(excluded.olton_percent_negative_direction,olton_percent_negative_direction),
    #         ole80 = coalesce(excluded.ole80,ole80),
    #         ole80_positive_direction = coalesce(excluded.ole80_positive_direction,ole80_positive_direction),
    #         ole80_negative_direction = coalesce(excluded.ole80_negative_direction,ole80_negative_direction),
    #         ole80_percent = coalesce(excluded.ole80_percent,ole80_percent),
    #         ole80_percent_positive_direction = coalesce(excluded.ole80_percent_positive_direction,ole80_percent_positive_direction),
    #         ole80_percent_negative_direction = coalesce(excluded.ole80_percent_negative_direction,ole80_percent_negative_direction),
    #         xe80 = coalesce(excluded.xe80,xe80),
    #         xe80_positive_direction = coalesce(excluded.xe80_positive_direction,xe80_positive_direction),
    #         xe80_negative_direction = coalesce(excluded.xe80_negative_direction,xe80_negative_direction),
    #         xe80_percent = coalesce(excluded.xe80_percent,xe80_percent),
    #         xe80_percent_positive_direction = coalesce(excluded.xe80_percent_positive_direction,xe80_percent_positive_direction),
    #         xe80_percent_negative_direction = coalesce(excluded.xe80_percent_negative_direction,xe80_percent_negative_direction),
    #         e80_per_day = coalesce(excluded.e80_per_day,e80_per_day),
    #         e80_per_day_positive_direction = coalesce(excluded.e80_per_day_positive_direction,e80_per_day_positive_direction),
    #         e80_per_day_negative_direction = coalesce(excluded.e80_per_day_negative_direction,e80_per_day_negative_direction),
    #         e80_per_heavy_vehicle = coalesce(excluded.e80_per_heavy_vehicle,e80_per_heavy_vehicle),
    #         e80_per_heavy_vehicle_positive_direction = coalesce(excluded.e80_per_heavy_vehicle_positive_direction,e80_per_heavy_vehicle_positive_direction),
    #         e80_per_heavy_vehicle_negative_direction = coalesce(excluded.e80_per_heavy_vehicle_negative_direction,e80_per_heavy_vehicle_negative_direction),
    #         worst_steering_single_axle_cnt = coalesce(excluded.worst_steering_single_axle_cnt,worst_steering_single_axle_cnt),
    #         worst_steering_single_axle_olhv_perc = coalesce(excluded.worst_steering_single_axle_olhv_perc,worst_steering_single_axle_olhv_perc),
    #         worst_steering_single_axle_tonperhv = coalesce(excluded.worst_steering_single_axle_tonperhv,worst_steering_single_axle_tonperhv),
    #         worst_steering_double_axle_cnt = coalesce(excluded.worst_steering_double_axle_cnt,worst_steering_double_axle_cnt),
    #         worst_steering_double_axle_olhv_perc = coalesce(excluded.worst_steering_double_axle_olhv_perc,worst_steering_double_axle_olhv_perc),
    #         worst_steering_double_axle_tonperhv = coalesce(excluded.worst_steering_double_axle_tonperhv,worst_steering_double_axle_tonperhv),
    #         worst_non_steering_single_axle_cnt = coalesce(excluded.worst_non_steering_single_axle_cnt,worst_non_steering_single_axle_cnt),
    #         worst_non_steering_single_axle_olhv_perc = coalesce(excluded.worst_non_steering_single_axle_olhv_perc,worst_non_steering_single_axle_olhv_perc),
    #         worst_non_steering_single_axle_tonperhv = coalesce(excluded.worst_non_steering_single_axle_tonperhv,worst_non_steering_single_axle_tonperhv),
    #         worst_non_steering_double_axle_cnt = coalesce(excluded.worst_non_steering_double_axle_cnt,worst_non_steering_double_axle_cnt),
    #         worst_non_steering_double_axle_olhv_perc = coalesce(excluded.worst_non_steering_double_axle_olhv_perc,worst_non_steering_double_axle_olhv_perc),
    #         worst_non_steering_double_axle_tonperhv = coalesce(excluded.worst_non_steering_double_axle_tonperhv,worst_non_steering_double_axle_tonperhv),
    #         worst_triple_axle_cnt = coalesce(excluded.worst_triple_axle_cnt,worst_triple_axle_cnt),
    #         worst_triple_axle_olhv_perc = coalesce(excluded.worst_triple_axle_olhv_perc,worst_triple_axle_olhv_perc),
    #         worst_triple_axle_tonperhv = coalesce(excluded.worst_triple_axle_tonperhv,worst_triple_axle_tonperhv),
    #         bridge_formula_cnt = coalesce(excluded.bridge_formula_cnt,bridge_formula_cnt),
    #         bridge_formula_olhv_perc = coalesce(excluded.bridge_formula_olhv_perc,bridge_formula_olhv_perc),
    #         bridge_formula_tonperhv = coalesce(excluded.bridge_formula_tonperhv,bridge_formula_tonperhv),
    #         gross_formula_cnt = coalesce(excluded.gross_formula_cnt,gross_formula_cnt),
    #         gross_formula_olhv_perc = coalesce(excluded.gross_formula_olhv_perc,gross_formula_olhv_perc),
    #         gross_formula_tonperhv = coalesce(excluded.gross_formula_tonperhv,gross_formula_tonperhv),
    #         total_avg_cnt = coalesce(excluded.total_avg_cnt,total_avg_cnt),
    #         total_avg_olhv_perc = coalesce(excluded.total_avg_olhv_perc,total_avg_olhv_perc),
    #         total_avg_tonperhv = coalesce(excluded.total_avg_tonperhv,total_avg_tonperhv),
    #         worst_steering_single_axle_cnt_positive_direciton = coalesce(excluded.worst_steering_single_axle_cnt_positive_direciton,worst_steering_single_axle_cnt_positive_direciton),
    #         worst_steering_single_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_steering_single_axle_olhv_perc_positive_direciton,worst_steering_single_axle_olhv_perc_positive_direciton),
    #         worst_steering_single_axle_tonperhv_positive_direciton = coalesce(excluded.worst_steering_single_axle_tonperhv_positive_direciton,worst_steering_single_axle_tonperhv_positive_direciton),
    #         worst_steering_double_axle_cnt_positive_direciton = coalesce(excluded.worst_steering_double_axle_cnt_positive_direciton,worst_steering_double_axle_cnt_positive_direciton),
    #         worst_steering_double_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_steering_double_axle_olhv_perc_positive_direciton,worst_steering_double_axle_olhv_perc_positive_direciton),
    #         worst_steering_double_axle_tonperhv_positive_direciton = coalesce(excluded.worst_steering_double_axle_tonperhv_positive_direciton,worst_steering_double_axle_tonperhv_positive_direciton),
    #         worst_non_steering_single_axle_cnt_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_cnt_positive_direciton,worst_non_steering_single_axle_cnt_positive_direciton),
    #         worst_non_steering_single_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_olhv_perc_positive_direciton,worst_non_steering_single_axle_olhv_perc_positive_direciton),
    #         worst_non_steering_single_axle_tonperhv_positive_direciton = coalesce(excluded.worst_non_steering_single_axle_tonperhv_positive_direciton,worst_non_steering_single_axle_tonperhv_positive_direciton),
    #         worst_non_steering_double_axle_cnt_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_cnt_positive_direciton,worst_non_steering_double_axle_cnt_positive_direciton),
    #         worst_non_steering_double_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_olhv_perc_positive_direciton,worst_non_steering_double_axle_olhv_perc_positive_direciton),
    #         worst_non_steering_double_axle_tonperhv_positive_direciton = coalesce(excluded.worst_non_steering_double_axle_tonperhv_positive_direciton,worst_non_steering_double_axle_tonperhv_positive_direciton),
    #         worst_triple_axle_cnt_positive_direciton = coalesce(excluded.worst_triple_axle_cnt_positive_direciton,worst_triple_axle_cnt_positive_direciton),
    #         worst_triple_axle_olhv_perc_positive_direciton = coalesce(excluded.worst_triple_axle_olhv_perc_positive_direciton,worst_triple_axle_olhv_perc_positive_direciton),
    #         worst_triple_axle_tonperhv_positive_direciton = coalesce(excluded.worst_triple_axle_tonperhv_positive_direciton,worst_triple_axle_tonperhv_positive_direciton),
    #         bridge_formula_cnt_positive_direciton = coalesce(excluded.bridge_formula_cnt_positive_direciton,bridge_formula_cnt_positive_direciton),
    #         bridge_formula_olhv_perc_positive_direciton = coalesce(excluded.bridge_formula_olhv_perc_positive_direciton,bridge_formula_olhv_perc_positive_direciton),
    #         bridge_formula_tonperhv_positive_direciton = coalesce(excluded.bridge_formula_tonperhv_positive_direciton,bridge_formula_tonperhv_positive_direciton),
    #         gross_formula_cnt_positive_direciton = coalesce(excluded.gross_formula_cnt_positive_direciton,gross_formula_cnt_positive_direciton),
    #         gross_formula_olhv_perc_positive_direciton = coalesce(excluded.gross_formula_olhv_perc_positive_direciton,gross_formula_olhv_perc_positive_direciton),
    #         gross_formula_tonperhv_positive_direciton = coalesce(excluded.gross_formula_tonperhv_positive_direciton,gross_formula_tonperhv_positive_direciton),
    #         total_avg_cnt_positive_direciton = coalesce(excluded.total_avg_cnt_positive_direciton,total_avg_cnt_positive_direciton),
    #         total_avg_olhv_perc_positive_direciton = coalesce(excluded.total_avg_olhv_perc_positive_direciton,total_avg_olhv_perc_positive_direciton),
    #         total_avg_tonperhv_positive_direciton = coalesce(excluded.total_avg_tonperhv_positive_direciton,total_avg_tonperhv_positive_direciton),
    #         worst_steering_single_axle_cnt_negative_direciton = coalesce(excluded.worst_steering_single_axle_cnt_negative_direciton,worst_steering_single_axle_cnt_negative_direciton),
    #         worst_steering_single_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_steering_single_axle_olhv_perc_negative_direciton,worst_steering_single_axle_olhv_perc_negative_direciton),
    #         worst_steering_single_axle_tonperhv_negative_direciton = coalesce(excluded.worst_steering_single_axle_tonperhv_negative_direciton,worst_steering_single_axle_tonperhv_negative_direciton),
    #         worst_steering_double_axle_cnt_negative_direciton = coalesce(excluded.worst_steering_double_axle_cnt_negative_direciton,worst_steering_double_axle_cnt_negative_direciton),
    #         worst_steering_double_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_steering_double_axle_olhv_perc_negative_direciton,worst_steering_double_axle_olhv_perc_negative_direciton),
    #         worst_steering_double_axle_tonperhv_negative_direciton = coalesce(excluded.worst_steering_double_axle_tonperhv_negative_direciton,worst_steering_double_axle_tonperhv_negative_direciton),
    #         worst_non_steering_single_axle_cnt_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_cnt_negative_direciton,worst_non_steering_single_axle_cnt_negative_direciton),
    #         worst_non_steering_single_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_olhv_perc_negative_direciton,worst_non_steering_single_axle_olhv_perc_negative_direciton),
    #         worst_non_steering_single_axle_tonperhv_negative_direciton = coalesce(excluded.worst_non_steering_single_axle_tonperhv_negative_direciton,worst_non_steering_single_axle_tonperhv_negative_direciton),
    #         worst_non_steering_double_axle_cnt_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_cnt_negative_direciton,worst_non_steering_double_axle_cnt_negative_direciton),
    #         worst_non_steering_double_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_olhv_perc_negative_direciton,worst_non_steering_double_axle_olhv_perc_negative_direciton),
    #         worst_non_steering_double_axle_tonperhv_negative_direciton = coalesce(excluded.worst_non_steering_double_axle_tonperhv_negative_direciton,worst_non_steering_double_axle_tonperhv_negative_direciton),
    #         worst_triple_axle_cnt_negative_direciton = coalesce(excluded.worst_triple_axle_cnt_negative_direciton,worst_triple_axle_cnt_negative_direciton),
    #         worst_triple_axle_olhv_perc_negative_direciton = coalesce(excluded.worst_triple_axle_olhv_perc_negative_direciton,worst_triple_axle_olhv_perc_negative_direciton),
    #         worst_triple_axle_tonperhv_negative_direciton = coalesce(excluded.worst_triple_axle_tonperhv_negative_direciton,worst_triple_axle_tonperhv_negative_direciton),
    #         bridge_formula_cnt_negative_direciton = coalesce(excluded.bridge_formula_cnt_negative_direciton,bridge_formula_cnt_negative_direciton),
    #         bridge_formula_olhv_perc_negative_direciton = coalesce(excluded.bridge_formula_olhv_perc_negative_direciton,bridge_formula_olhv_perc_negative_direciton),
    #         bridge_formula_tonperhv_negative_direciton = coalesce(excluded.bridge_formula_tonperhv_negative_direciton,bridge_formula_tonperhv_negative_direciton),
    #         gross_formula_cnt_negative_direciton = coalesce(excluded.gross_formula_cnt_negative_direciton,gross_formula_cnt_negative_direciton),
    #         gross_formula_olhv_perc_negative_direciton = coalesce(excluded.gross_formula_olhv_perc_negative_direciton,gross_formula_olhv_perc_negative_direciton),
    #         gross_formula_tonperhv_negative_direciton = coalesce(excluded.gross_formula_tonperhv_negative_direciton,gross_formula_tonperhv_negative_direciton),
    #         total_avg_cnt_negative_direciton = coalesce(excluded.total_avg_cnt_negative_direciton,total_avg_cnt_negative_direciton),
    #         total_avg_olhv_perc_negative_direciton = coalesce(excluded.total_avg_olhv_perc_negative_direciton,total_avg_olhv_perc_negative_direciton),
    #         total_avg_tonperhv_negative_direciton = coalesce(excluded.total_avg_tonperhv_negative_direciton,total_avg_tonperhv_negative_direciton)
    #     )
    # """
    break

In [13]:
GVML = pd.read_sql_query(queries.GET_GROSS_VEHICLE_MASS_LIMITS, config.ENGINE)
AXLL = pd.read_sql_query(queries.GET_AXLE_LOAD_LIMITS, config.ENGINE)
header_ids = pd.read_sql_query(GET_HSWIM_HEADER_IDS, config.ENGINE)

In [54]:
for header_id in list(header_ids['header_id'].astype(str)):
    SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY = wim_stations_header_insert_qrys(header_id)
    df, df2, df3 = wim_stations_header_insert_dfs(SELECT_TYPE10_QRY, AXLE_SPACING_SELECT_QRY, WHEEL_MASS_SELECT_QRY)
    if df2 is None or df3 is None:
        pass
    else:
        insert_string = wim_stations_header_insert(header_id, df, df2, df3)
        with config.ENGINE.connect() as conn:
            conn.execute(insert_string)
            print('COMPLETE')

COMPLETE
COMPLETE
COMPLETE
COMPLETE


In [39]:
df.loc[(df['group']=='Heavy')&(df['direction']=='N')].count()[0]

38640

In [44]:
round(((df3.loc[df3['direction']=='N']['wheel_mass']*2).sum()/1000)/df.loc[df['group']=='Heavy'].count()[0],2)

20.67